# Convert Lat/Long to Block Number

Import libraries

In [11]:
import pandas as pd
import sys
import time
import urllib.request
import re

Read in arbitrary dataframe for testing

In [5]:
racks = pd.read_csv('data/bicycle_rack_points.csv')

Function to scrape FCC website for block number by lat/long. Just insert a dataframe with lat and long labeled as Y and X columns, respectively. It'll output a pandas Series of block numbers (you'll need to manually tack it on to the dataframe).

In [8]:
def lltoblock(df):

    # Create empty array for storing block number
    block_list = pd.Series([])

    # Create opener
    opener = urllib.request.FancyURLopener({})

    if 'X' in df.columns and 'Y' in df.columns:
        lat = df['Y']
        lng = df['X']
    elif 'Latitude' in df.columns and 'Longitude' in df.columns:
        lat = df['Latitude']
        lng = df['Longitude']
    elif 'Meter_Lat' in df.columns and 'Meter_Long' in df.columns:
        lat = df['Meter_Lat']
        lng = df['Meter_Long']
    else:
        lat = None
        lng = None


    # Iterate over all rows of dataframe
    for i in range(len(df)):

        # Create url from lat/long
        url = 'https://geo.fcc.gov/api/census/block/find?latitude='\
        + str(lat[i]) + '&longitude=' + str(lng[i]) + '&showall=false&format=json'

        # Store webpage contents
        f = opener.open(url)
        content = f.read()

        # Get block number out of content string
        block = re.search(r".*FIPS\":\" *(.*?) *\",\"bbox.*", str(content)).group(1)

        # Add current block number to our block array
        block_list[i] = block

    # Return the list of block numbers
    return block_list

Call the function on a test dataframe

In [9]:
lltoblock(racks).head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  import sys


0    515400002021030
1    515400007003009
2    515400010001056
3    515400002022008
4    515400002021030
dtype: object

In [10]:
lltoblock(pd.read_csv('./data/parking_meter_pilot_data_.csv').iloc[:5,:])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  import sys


0    515400010001043
1    515400010001043
2    515400010001043
3    515400010001043
4    515400010001043
dtype: object

In [62]:
def addresstoll(df):
   
    # Create empty lists for lats and lngs
    lat_list = pd.Series([])
    lng_list = pd.Series([])    
    
    # Create opener
    opener = urllib.request.FancyURLopener({})
        
    # Iterate over all rows of dataframe
    for i in range(len(df)):
        
        # Get block number (street number) and street name
        block = df['BlockNumber'][i]
        street = df['StreetName'][i]
        
        # Create url from these
        url = 'http://www.datasciencetoolkit.org/street2coordinates/'        + str(block) + '+' + str(street) + '+' + 'Charlottesville+VA'
        
        # Store webpage contents
        f = opener.open(url)
        content = f.read()
        
        # Get lat and long out of content string
        lat = re.search(r".*latitude\": *(.*?) *,\n*", str(content)).group(1)
        lng = re.search(r".*longitude\": *(.*?) *,\n*", str(content)).group(1)
        
        # Add to the lists
        lat_list[i] = lat
        lng_list[i] = lng
               
    return lat_list, lng_list
#       sys.stdout.write('\r' + str(lat) +', '+ str(lng))
#       time.sleep(0.1)

In [64]:
crime = pd.read_csv('./data/Crime_Data.csv')

In [66]:
crime[:5]

,RecordID,Offense,IncidentID,BlockNumber,StreetName,Agency,DateReported,HourReported
0,1001,Larceny - All Other,201400000700,600.0,EVERGREEN AVE,CPD,2014-02-07T17:31:00.000Z,1231
1,1002,Larceny - From Motor Vehicle,201400000701,600.0,9TH ST SW,CPD,2014-02-07T19:01:00.000Z,1401
2,1003,Larceny - All Other,201400000702,1700.0,CHESAPEAKE ST,CPD,2014-02-07T21:28:00.000Z,1628
3,1004,Vandalism,201400000703,300.0,E HIGH ST,CPD,2014-02-07T21:48:00.000Z,1648
4,1005,Lost/FoundProperty,201400000704,1300.0,PRESTON AVE,CPD,2014-02-07T20:05:00.000Z,1505


In [67]:
crime_lats, crime_lngs = addresstoll(crime[:5])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  


In [68]:
crime_lats

0    38.037375
1    38.025228
2    38.022897
3    38.033223
4    38.044213
dtype: object

In [69]:
crime_lngs

0    -78.470562
1    -78.495113
2    -78.455618
3    -78.483328
4    -78.492964
dtype: object